#Step 1 Starting Point As Per Sample Notebook

In [1]:
!pip install pathway bokeh panel --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 📥 SECTION 2: Data Preparation

In [2]:
import pandas as pd

df = pd.read_csv("dataset.csv")

# Combine date and time
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                 format='%d-%m-%Y %H:%M:%S')

# Vehicle weights with default 1.0
vehicle_weights = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
df['vehicle_weight'] = (
    df['VehicleType'].map(vehicle_weights).fillna(1.0).astype(float)
)

# ✅ Traffic mapping with enforced int via lambda
traffic_map = {"low": 1, "medium": 2, "high": 3}
df['traffic_level'] = df['TrafficConditionNearby'].map(traffic_map)
df['traffic_level'] = df['traffic_level'].fillna(0).apply(lambda x: int(x))

# Rename & select columns
df_stream = df.rename(columns={
    'SystemCodeNumber': 'lot_id',
    'Occupancy': 'occupancy',
    'Capacity': 'capacity',
    'QueueLength': 'queue_length',
    'IsSpecialDay': 'is_special_day'
})[[
    'Timestamp', 'lot_id', 'occupancy', 'capacity',
    'queue_length', 'traffic_level', 'is_special_day', 'vehicle_weight'
]]

# Export cleaned data
df_stream.to_csv("parking_stream.csv", index=False)

## 🧠 Demand Function Explanation
Our demand function is a weighted sum of five real-time features:
- **Occupancy ratio** (occupancy / capacity): how full the lot is
- **Queue length**: number of vehicles waiting
- **Traffic level**: traffic near the lot (mapped from low/medium/high)
- **Special days**: whether the day is a public/known holiday (0/1)
- **Vehicle weight**: demand proxy based on vehicle type (car/truck/bike)

Weights are selected based on intuitive impact on demand, scaled and normalized.

## ⚖️ Assumptions & Weights Justification
- `W["occupancy"] = 0.4`: Occupancy is the most direct measure of demand.
- `W["queue"] = 0.3`: A longer queue reflects unmet demand.
- `W["traffic"] = 0.15`: More traffic means more potential drivers.
- `W["special"] = 0.1`: Holidays or events increase parking pressure.
- `W["vehicle"] = 0.05`: Heavy vehicles are weighted more in demand.

Weights sum to 1.0 for normalized interpretation.

## 💰 Dynamic Pricing Formula Used
Final price is determined using the normalized demand:
```python
price = normalized_demand * LAMBDA * BASE + BASE
```
- `BASE` = 10.0: baseline price when demand is 0
- `LAMBDA` = 0.5: scaling factor to adjust sensitivity

Prices increase smoothly with demand while remaining interpretable.

## 📡 SECTION 3: Pathway Streaming Setup

In [3]:
import pathway as pw

# Define the schema for the streaming data using Pathway
# This schema specifies the expected structure of each data row in the stream
class ParkingSchema(pw.Schema):
    Timestamp: str
    lot_id: str
    occupancy: int
    capacity: int
    queue_length: int
    traffic_level: float
    is_special_day: int
    vehicle_weight: float

# Stream the data
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming
# input_rate=1000 means approximately 1000 rows per second will be ingested into the stream.
stream = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)


In [4]:
from datetime import timedelta
import pathway as pw

# Define the datetime format to parse the 'Timestamp' column
# Parse timestamp and derive daily instance key
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data = stream.with_columns(
    t = stream.Timestamp.dt.strptime(fmt),
    day = stream.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


# 🧩 Model 1: Baseline Linear Pricing

In [5]:
ALPHA = 1.5
BASE_PRICE = 10.0

# Define a tumbling window of 1 day
windowed_model1 = data.windowby(
    pw.this.t,
    instance=pw.this.lot_id + "_" + pw.this.day,
    window=pw.temporal.tumbling(timedelta(days=1)),
    behavior=pw.temporal.exactly_once_behavior()
)

# Calculate the price for Model 1
model1_price = windowed_model1.reduce(
    t=pw.this._pw_window_end,
    lot_id=pw.reducers.max(pw.this.lot_id),
    occ_sum=pw.reducers.sum(pw.this.occupancy),
    count=pw.reducers.count(),
    cap=pw.reducers.max(pw.this.capacity),
).with_columns(
    # First, calculate the average occupancy in one call
    occ_avg=pw.this.occ_sum / pw.this.count
).with_columns(
    # Then, use the new column in a second call to calculate the price
    price_model_1=BASE_PRICE + ALPHA * (pw.this.occ_avg / pw.this.cap)
)

## 🧠 SECTION 4: Real-Time Pricing Logic (Model 2)

In [6]:

# Define demand weights and price scaling
W = {"occupancy": 0.4, "queue": 0.3, "traffic": 0.15, "special": 0.1, "vehicle": 0.05}
LAMBDA = 0.5
BASE = 10.0

# Tumbling daily window by lot
windowed = data.windowby(
    pw.this.t,
    instance=pw.this.lot_id + "_" + pw.this.day,
    window=pw.temporal.tumbling(timedelta(days=1)),
    behavior=pw.temporal.exactly_once_behavior()
)

# Reduce to daily aggregates
windowed_price = windowed.reduce(
    t = pw.this._pw_window_end,
    occ_max = pw.reducers.max(pw.this.occupancy),
    cap = pw.reducers.max(pw.this.capacity),
    queue_sum = pw.reducers.sum(pw.this.queue_length),
    traffic_sum = pw.reducers.sum(pw.this.traffic_level),
    vehicle_sum = pw.reducers.sum(pw.this.vehicle_weight),
    count = pw.reducers.count(),
    special = pw.reducers.max(pw.this.is_special_day),
    lot_id = pw.reducers.max(pw.this.lot_id)
)

# Compute averages (no .cast() needed)
windowed_price = windowed_price.with_columns(
    queue_avg = pw.this.queue_sum / pw.this.count,
    traffic_avg = pw.this.traffic_sum / pw.this.count,
    vehicle_avg = pw.this.vehicle_sum / pw.this.count
)

# Step 1: Compute demand
windowed_price = windowed_price.with_columns(
    demand = (
        W["occupancy"] * (pw.this.occ_max / pw.this.cap) +
        W["queue"] * pw.this.queue_avg +
        W["traffic"] * pw.this.traffic_avg +
        W["special"] * pw.this.special +
        W["vehicle"] * pw.this.vehicle_avg
    )
)

# Step 2: Normalize demand
windowed_price = windowed_price.with_columns(
    normalized_demand = pw.this.demand / 5.0
)

# Step 3: Compute price
windowed_price = windowed_price.with_columns(
    price = pw.this.normalized_demand * LAMBDA * BASE + BASE
)


## 📊 SECTION 5: Bokeh Real-Time Visualization

In [7]:
import bokeh.plotting
import panel as pn

In [8]:
# Model 1 Plot

# Activate Panel extension for interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function for Model 1
def plot_model1(source):
    fig = bokeh.plotting.figure(
        width=800,
        height=400,
        x_axis_type="datetime",
        title="Model 1: Baseline Linear Pricing"
    )
    # Line plot
    fig.line("t", "price_model_1", source=source, line_width=2, color="gray", legend_label="Model 1")
    # Scatter points for better visibility
    fig.scatter("t", "price_model_1", source=source, size=5, color="black")

    fig.legend.location = "top_left"
    return fig

# Generate the Bokeh plot using Pathway's .plot()
viz_model1 = model1_price.plot(plot_model1, sorting_col="t")

# Display using Panel layout
pn.Column(viz_model1).servable()



Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [9]:
# Model 2 Plot

# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def plotter(source):
    fig = bokeh.plotting.figure(
        width=800,
        height=400,
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
        title="Model 2: Real Time Demand-Based Pricing"
    )
    fig.line("t", "price", source=source, line_width=2, color="green")
    fig.scatter("t", "price", source=source, size=5, color="blue")  # ✅ Bokeh 3.4-compatible
    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = windowed_price.plot(plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [10]:
df = pd.read_csv("parking_stream.csv")
lot_ids = df["lot_id"].unique().tolist()
print(lot_ids)

['BHMBCCMKT01', 'BHMBCCTHL01', 'BHMEURBRD01', 'BHMMBMMBX01', 'BHMNCPHST01', 'BHMNCPNST01', 'Broad Street', 'Others-CCCPS105a', 'Others-CCCPS119a', 'Others-CCCPS135a', 'Others-CCCPS202', 'Others-CCCPS8', 'Others-CCCPS98', 'Shopping']


In [11]:
import panel as pn
import bokeh.plotting

lot_viz_panels = []

def create_plot_for_lot(lot_id):
    filtered_lot = windowed_price.filter(pw.this.lot_id == lot_id)

    def plot_fn(source):
        fig = bokeh.plotting.figure(
            width=800, height=300, x_axis_type="datetime",
            title=f"Real-Time Pricing for Lot: {lot_id}"
        )
        fig.line("t", "price", source=source, color="blue", legend_label=f"Lot {lot_id}")
        fig.legend.location = "top_left"
        return fig

    return pn.panel(filtered_lot.plot(plot_fn, sorting_col="t"))

# Assuming you already have lot_ids = ["L001", "L002", ...]
for lot in lot_ids:
    lot_viz_panels.append(create_plot_for_lot(lot))

pn.Column(*lot_viz_panels).servable()



Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [1] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [2] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [3] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [4] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [5] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [6] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [7] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [8] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [9] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [10] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [11] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [12] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [13] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

## 🏃 SECTION 6: Run the Pathway Pipeline

In [12]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

# %%capture --no-display
pw.run()


Output()